In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/schiend/Desktop/DOHERTY/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [2]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/wEsnzt/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/A862hn/')
im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'A862hn')

In [3]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

Could not find Java JRE compatible with x86_64 architecture


In [ ]:
channel_names = [
    'BCL2','CD10','Hoechst 33342','CD20','CD3','Collagen IV','CD23','BCL6','CD138','IgD','CD31','CD21','CD163','HLA-DR','CD11c','CD1c','CD25','FOXP3','CD8','CD69','PD-1','CD4','CD38','SPARC','CD68','IRF4','ICOS','Ki-67','CD35','CXCL13','TCR Va7.2','CD44','CD106','a-SMA','Lyve-1','DC-SIGN','CD45','CLEC9a AF488'
]

In [4]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = False,
    visible = False, squeeze = False
)

In [ ]:
napari_utils.show_labels_all(
    show_labels = True,
    show_points = True,
    value_names = ['default']
)

In [ ]:
# show populations
napari_utils.show_pop_mapping(
    "clust", value_name = "default")